In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone,ServerlessSpec

In [6]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",dimension=1536,metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '36373c6dc217c9ea89de4a34ac67424a', 'Date': 'Mon, 16 Sep 2024 03:51:39 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [9]:
import json
data=json.load(open("reviews.json"))

In [11]:
processed_data = []
client = OpenAI()

# Create embeddings for each review
for i, review in enumerate(data["reviews"]):
    print(f"Processing review {i+1} of {len(data['reviews'])}")
    try:
        response = client.embeddings.create(
            input=review['review'], model="text-embedding-3-small"
        )
        embedding = response.data[0].embedding
        processed_data.append(
            {
                "values": embedding,
                "id": review["professor"],
                "metadata": {
                    "review": review["review"],
                    "subject": review["subject"],
                    "stars": review["stars"],
                }
            }
        )
    except Exception as e:
        print(f"Error processing review {i+1}: {e}")


Processing review 1 of 20
Processing review 2 of 20
Processing review 3 of 20
Processing review 4 of 20
Processing review 5 of 20
Processing review 6 of 20
Processing review 7 of 20
Processing review 8 of 20
Processing review 9 of 20
Processing review 10 of 20
Processing review 11 of 20
Processing review 12 of 20
Processing review 13 of 20
Processing review 14 of 20
Processing review 15 of 20
Processing review 16 of 20
Processing review 17 of 20
Processing review 18 of 20
Processing review 19 of 20
Processing review 20 of 20


In [12]:
processed_data[0]

{'values': [0.03084503,
  -0.0146045545,
  0.050766326,
  0.032454565,
  0.0008657849,
  0.0069592614,
  0.011253554,
  0.056096263,
  -0.004884676,
  0.019380389,
  0.028866094,
  -0.019525511,
  -0.02278416,
  0.034776516,
  0.004581239,
  0.015000341,
  -0.030185385,
  -0.026399018,
  0.04461843,
  0.07118896,
  0.03651798,
  -0.024301345,
  0.01602939,
  -0.028021747,
  -0.022454338,
  -0.04108273,
  0.017652117,
  0.019762984,
  0.038787164,
  0.0006575842,
  0.074249715,
  -0.006563474,
  -0.03812752,
  -0.00069922436,
  -0.021623183,
  0.036386054,
  0.026491368,
  -0.019314423,
  0.023061212,
  0.0052837613,
  0.0050034123,
  -0.012684985,
  -0.02522485,
  -0.008331324,
  0.048708234,
  0.0020366558,
  0.014644133,
  -0.015528059,
  0.04625435,
  0.05535746,
  -0.007869572,
  0.00090536365,
  0.014789255,
  -0.04266588,
  -0.01107545,
  -0.038760778,
  -0.0070582083,
  0.04013284,
  -0.0027985466,
  -0.022836931,
  0.04018561,
  -0.0008723813,
  -0.01816664,
  -0.013918523,
  -

In [13]:
index=pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace='ns1'
)

{'upserted_count': 20}

In [14]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}